# Hopfield Network

Import dependencies

In [1]:
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

/home/philipp/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
TRAIN_DIR = Path('/home/philipp/Projects/Hopfield-Neural-Network/mnist_png/train/')
TEST_DIR = Path('/home/philipp/Projects/Hopfield-Neural-Network/mnist_png/test/')

Train files:

In [3]:
train_files = sorted(list(TRAIN_DIR.rglob('*.png')))
train_labels = [int(path.name[:1]) for path in train_files]
train_images = []
for file in train_files:
    train_images.append(Image.open(file))

Test files:

In [64]:
test_files = []
test_labels = []
test_images = []
for folder in sorted(TEST_DIR.iterdir()):
    for file in sorted(list(folder.rglob('*.png')))[:1]:
        test_images.append(Image.open(file))
        test_labels.append(int(file.parent.name))

### Util functions

### Pre-Process Data

In [5]:
def prepare_data(images, labels):
    data = {}
    for im, lab in zip(images, labels):
        image = np.array(im).flatten()
        image = [1 if pixel > 0 else -1 for pixel in image]
        data[lab] = np.array(image)
        data[lab].resize((1,28*28))
    return data

In [6]:
train_data = prepare_data(train_images, train_labels)

In [7]:
test_data = prepare_data(test_images, test_labels)

##### Data Parameters

In [8]:
n_train = len(train_data)

In [9]:
# Size of image(width)
n_side = 28

# Number of neurons
n_neurons = n_side * n_side

### Neural Network

Function to train the network using Hebbian learning rule

In [11]:
def train(neu, train_data):
    w = np.zeros([neu, neu])
    for index in train_data:
        w += 4 * np.outer(train_data[index], train_data[index])
        im_i = np.ones((n_side*n_side, 1)) @ train_data[index]
        im_j = train_data[index].T @ np.ones((1,28*28)) 
        w -= 2 * im_i
        w -= 2 * im_j   
        w -= 1
    for diag in range(neu):
        w[diag][diag] = 0
    return w

In [48]:
def train(neu, train_data):
    w = np.zeros([neu, neu])
    for x in range(len(train_data)):
        for i in range(neu):
            for j in range(neu):
                w[i][j] += train_data[x][0][i] * train_data[x][0][j]
                if i == j:
                    w[i, j] = 0
    return w

Function to test the network

In [12]:
def test(weights, train_data, test_data, theta=0):
    accuracy = 0.0
    for label in test_data:
        image = test_data[label]
        predict_data = retrieve_pattern(weights, image, theta)
        if np.array_equal(train_data[label], predict_data):
            accuracy += 1.0
    accuracy /= len(test_data)
    return accuracy

In [45]:
def retrieve_pattern(weights, data, theta=0, steps=100):

    res = data
    for _ in range(steps):
        res = res @ weights
        for i in range(len(res[0])):
            if res[0][i] > theta:
                res[0][i] = 1
            else:
                res[0][i] = -1
    return res

In [24]:
def show_image(image):
    image_2d = image.copy()
    image_2d.resize((28,28))
    image_2d = Image.fromarray(np.uint8(255*image_2d))
    display(image_2d)

In [57]:
def get_output_neuron(w, u):
    lb = 10000
    neu = 28*28
    for l in range(lb):
        neuron = randint(0,neu-1)
        c = 0
        for i in range(0, neu-1):
            c += u[0][i] * w[i][neuron]
        if c > 0:
            u[0][neuron] = 1
        elif c < 0:
            u[0][neuron] = -1
        else:
            u[0][neuron] = 0
    return u[0]

In [23]:
def test(weights, train_data, test_data, theta=0):
    accuracy = 0.0
    for label in test_data:
        image = test_data[label]
        predict_data = get_output_neuron(weights, image)
        if np.array_equal(train_data[label], predict_data):
            accuracy += 1.0
    accuracy /= len(test_data)
    return accuracy

#### Train

In [50]:
from random import randint

In [78]:
W = train(n_neurons, train_data)

#### Test

In [58]:
accuracy = test(W, train_data, test_data)

Print accuracy

In [59]:
print("Accuracy of the network is %f" % (accuracy * 100))

Accuracy of the network is 0.000000


In [80]:
pred = get_output_neuron(W, train_data[0])
show_image(pred)